In [ ]:
"""Time series BC plot at qoms site"""
import matplotlib
from dateutil.parser import parse
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.dates as mdates
import matplotlib.ticker as tck
import matplotlib.patches as mpatches

from matplotlib import rcParams

"""Control font"""
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['font.sans-serif'] = "Times New Roman"

#==== Add cyberpunk effect =====
#import mplcyberpunk
#plt.style.use("cyberpunk")
#plt.style.use("dark_background")
#mplcyberpunk.add_glow_effects()
#mplcyberpunk.make_lines_glow()
#====================================================================
#df1 = pd.read_csv(r'/mnt/g/Paper_work/modelVsobservation_BC.csv')
df1  = pd.read_excel(r'/mnt/g/Paper_work/modelVsobservation_BC_new.xlsx',sheet_name='Sheet1')
df1['observed']=pd.to_numeric(df1['observed'],errors='coerce')
df1['model']=pd.to_numeric(df1['model'],errors='coerce')
df1 = df1.dropna()
#date = df1['date']
#obs1 = df1['observed'].astype(float)
df = df1.groupby(df1["date"].dt.date)["observed"].mean().to_frame()
df2 =df1.groupby(df1["date"].dt.date)["model"].mean().to_frame()

#df['observed']=pd.to_numeric(df['observed'],errors='coerce')
#df['model']=pd.to_numeric(df2['model'],errors='coerce')

observed_lower = df.observed*0.85
observed_upper = df.observed*1.15

mod_lo = df2.model*0.85
mod_up = df2.model*1.15


""" Figure"""

fig = plt.figure(figsize = (10, 6),dpi=200)
ax = fig.add_subplot()
#fig.autofmt_xdate()

#mplcyberpunk.make_lines_glow(ax)

#==observation
#ax.plot(df.observed, color = "lightcoral", lw = 0.3,alpha=1)
obs = ax.plot(observed_lower, color = "lightslategray",alpha=0.8)
ax.plot(observed_upper, color = "lightslategray",alpha=0.8)
ax.plot(df.observed, color = "lightcoral", lw = 0.3,alpha=0.8)
#===plot model
#ax.plot(df2.model, color = "black", lw = 0.5,alpha=0.8)
mod = ax.plot(mod_lo, color = "skyblue",alpha=0.8)
ax.plot(mod_up,color = "skyblue",alpha=0.8)
ax.plot(df2.model, color = "black", lw = 0.3,alpha=0.8)

# fill the area between the 3 lines
ax.fill_between(df.observed.index, df.observed, observed_lower, where = df.observed > observed_lower, 
								facecolor='lightslategray', interpolate = True,zorder=0,alpha=0.8)
ax.fill_between(df.observed.index, df.observed, observed_upper, where = observed_upper > df.observed, 
								facecolor='lightslategray', interpolate = True,zorder=0,alpha=0.8)


ax.fill_between(df2.model.index, df2.model, mod_lo, where = df2.model > mod_lo, facecolor='skyblue', interpolate = True,alpha=0.8,zorder=1)
ax.fill_between(df2.model.index, df2.model, mod_up, where = mod_up > df2.model, facecolor='skyblue', interpolate = True,alpha=0.8,zorder=1)

#==== highlight desired region =====
ax.axvspan(*mdates.datestr2num(['04/01/2017', '04/25/2017']), color='silver', alpha=0.5)

ax.set_ylim(0, 4)
ax.set_xlabel("Date",fontsize = 10)
ax.set_ylabel("BC  [$µg  m^{-3}$]",fontsize = 10)

# get the xticks and the xticks labels
xtick_location = df.index.tolist()[::30]
xtick_labels = df.index.tolist()[::30]

# set the xticks to be every 2'th entry
# every 2 months
ax.set_xticks(xtick_location)
ax.set_xticklabels(xtick_labels, fontdict={'horizontalalignment': 'center', 'verticalalignment': 'center_baseline', "fontsize":"12"})
#==== Add minor tick label =====
ax.yaxis.set_minor_locator(tck.AutoMinorLocator())
#ax.xaxis.set_minor_locator(tck.AutoMinorLocator())
# change the x and y tick size
ax.tick_params(axis = 'x', labelsize = 10)
ax.tick_params(axis = 'y', labelsize = 10)

# add a title and a gridline
ax.set_title("BC Mean orders ± 15% interval", fontsize = 10,pad=0)
ax.grid(axis = "y", alpha = .3)


#=== Insert legend
#ax.legend(loc='upper right', bbox_to_anchor=(1.35, 0.8),fontsize=4)
#ax.legend([obs, mod], ['Observation','Model'])
obs_patch = mpatches.Patch(color='lightslategray', label='Observation BC')
mod_patch = mpatches.Patch(color='skyblue', label='Model BC')

plt.legend(handles=[obs_patch, mod_patch],fontsize = 10)


#==== add equation ========
bbox = dict(boxstyle="round", fc="0.94")
offset = 72
xdata, ydata = *mdates.datestr2num(['2017-03-10']),2.9
ax.annotate(
    f'$R^2 = 0.30$' '\ny=1.358x+159.85',
    (xdata, ydata),size=10,
    xytext=(-2*offset, offset), textcoords='offset points',
    bbox=bbox) 



#===figure properties
fig.subplots_adjust(top=0.95,
                        bottom=0.095,
                        left=0.09,
                        right=0.99,
                        hspace=0.2,
                        wspace=0.2)



#plt.savefig("/mnt/g/2nd_Paper/ModelVsObs.png",dpi=900)
plt.show()

